# Завдання
Використовуючи тренувальний датасет з архіву `data.zip` та матеріалів з додаткових джерел, побудуйте систему детекції медичних масок на обличчі.

## Рішення

In [2]:
import splitfolders

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import transforms

from torchvision import datasets
from torch.utils.data import DataLoader

from torch.utils.tensorboard import SummaryWriter

import warnings
warnings.filterwarnings("ignore")

In [5]:
# Розбиття на тренувальну та тестову вибірки
splitfolders.ratio(input='img', output="data", seed=2023, ratio=(.8, 0, 0.2))

Copying files: 1376 files [00:00, 2032.26 files/s]


In [4]:
# визначення трансформацій зображень

image_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize(size=(224,224)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test':
    transforms.Compose([
        transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

In [5]:
# Завантаження даних
data = {
    'train':
    datasets.ImageFolder(root='./data/train', transform=image_transforms['train']),
    'test':
    datasets.ImageFolder(root='./data/test', transform=image_transforms['test']),
}

# Завантаження даних в даталоадери
dataloaders = {
    'train': DataLoader(data['train'], batch_size=100, shuffle=True),
    'test': DataLoader(data['test'], batch_size=100, shuffle=True)
}

In [3]:
# завантаження MobileNetv2
model = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)

Using cache found in C:\Users\ihorm/.cache\torch\hub\pytorch_vision_v0.6.0


In [7]:
# фіксація початкових шарів MobileNetv2
for param in model.parameters():
    param.requires_grad = False

In [8]:
# додавання власного класифікатора
model.classifier[1] = nn.Sequential(
                      nn.Linear(1280, 256),
                      nn.ReLU(),
                      nn.Linear(256, 128),
                      nn.ReLU(),
                      nn.Dropout(0.4),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 32),
                      nn.ReLU(),
                      nn.Dropout(0.4),
                      nn.Linear(32, 2),
                      nn.LogSoftmax(dim=1))

In [9]:
# перевірка тренуваних параметрів
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

371234


In [10]:
# перевірка доступності GPU
if torch.cuda.is_available():
    print('training on GPU')
    device = torch.device("cuda:0")
else:
    print('training on CPU')
    device = torch.device("cpu")

training on GPU


In [11]:
# Навчання
from tqdm.notebook import tqdm # для детекції виконання

def train(model, optimizer, loss_fn, train_loader, val_loader, epochs,device):
    writer = SummaryWriter("saves/logs")
    for epoch in tqdm(range(epochs)):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in (train_loader):
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)

        model.eval()
        num_correct = 0
        num_examples = 0
        for batch in (val_loader):
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets)
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}, accuracy = {:.4f}'.format(epoch, training_loss,valid_loss, num_correct / num_examples))

        # збереження моделі
        if(training_loss<0.005):
            torch.save(model,"saves/"+str(epoch)+".pth")

        # tensorBoard logs
        writer.add_scalar('Loss', loss.item(), epoch)
        writer.add_scalar('Train/Loss', training_loss, epoch)
        writer.add_scalar('Test/Loss', valid_loss, epoch)
        writer.add_scalar('Test/Accuracy',num_correct / num_examples, epoch)
        writer.flush()

In [12]:
# Тестування
def test_model(model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloaders['test']:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('correct: {:d}  total: {:d}'.format(correct, total))
    print('accuracy = {:f}'.format(correct / total))

In [13]:
# Передача моделі у GPU
model=model.to(device)

In [14]:
# вибір оптимізатора та функції втрат
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss = torch.nn.CrossEntropyLoss()

In [15]:
# навчання моделі
train(model, optimizer,loss,dataloaders['train'],dataloaders['test'],100,device)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0, Training Loss: 0.6017, Validation Loss: 0.3250, accuracy = 0.9710
Epoch: 1, Training Loss: 0.2110, Validation Loss: 0.0402, accuracy = 0.9855
Epoch: 2, Training Loss: 0.0564, Validation Loss: 0.0254, accuracy = 0.9891
Epoch: 3, Training Loss: 0.0464, Validation Loss: 0.0262, accuracy = 0.9928
Epoch: 4, Training Loss: 0.0486, Validation Loss: 0.0253, accuracy = 0.9928
Epoch: 5, Training Loss: 0.0612, Validation Loss: 0.0859, accuracy = 0.9638
Epoch: 6, Training Loss: 0.0295, Validation Loss: 0.0254, accuracy = 0.9928
Epoch: 7, Training Loss: 0.0552, Validation Loss: 0.0249, accuracy = 0.9855
Epoch: 8, Training Loss: 0.0181, Validation Loss: 0.0333, accuracy = 0.9891
Epoch: 9, Training Loss: 0.0387, Validation Loss: 0.0365, accuracy = 0.9928
Epoch: 10, Training Loss: 0.0185, Validation Loss: 0.0356, accuracy = 0.9855
Epoch: 11, Training Loss: 0.0207, Validation Loss: 0.0386, accuracy = 0.9928
Epoch: 12, Training Loss: 0.0113, Validation Loss: 0.0234, accuracy = 0.9891
Epoch: 13

In [16]:
# точність моделі
test_model(model)

correct: 273  total: 276
accuracy = 0.989130
